In [27]:
import os
import xarray as xr
import pandas as pd
import numpy as np
from datetime import datetime

def load_data_at_altitude(altitude, data_folder, latitudes, longitudes):
    # List all files in the folder
    file_list = os.listdir(data_folder)

    # Filter the files to keep only those with the correct naming convention
    filtered_files = [file for file in file_list if file.endswith('_mfx.npz') and '2018-02' in file]

    # Sort the files by date
    filtered_files.sort()

    # Load all files into a single xarray Dataset
    data_arrays = [xr.DataArray(np.load(f'{data_folder}/{file}')['arr_0'], 
                                coords={
                                'latitude': (('latitude'), latitudes),
                                'longitude': (('longitude'), longitudes),
                                'time': datetime.strptime(file[:10], '%Y-%m-%d'),
                                'altitude': altitude}, 
                                dims=('latitude', 'longitude'),
                                attrs={'altitude': altitude}) 
                   for file in filtered_files]
    combined_data = xr.concat(data_arrays, dim='time')
    return combined_data

# Set the folder paths
data_folder_24km = "processed_airs_sftp/40_km_grid/24km/mfx"
data_folder_30km = "processed_airs_sftp/40_km_grid/30km/mfx/mfx_2009_jan/"
data_folder_36km = "processed_airs_sftp/40_km_grid/36km/mfx"
data_folder_42km = "processed_airs_sftp/40_km_grid/42km/mfx"
# lat = "coordinates_40km_grid/Airs_nh_lat_40_grid.csv"
# lon = "coordinates_40km_grid/Airs_nh_lon_40_grid.csv"

lat = 'lat.csv'
lon = 'lon.csv'

# Read the latitude and longitude data from the CSV file
latitudes = np.loadtxt(lat, delimiter=',', unpack=True)
longitudes = np.loadtxt(lon, delimiter=',', unpack=True)

# Load data at 30km and 40km altitudes
data_24km = load_data_at_altitude(24, data_folder_24km, latitudes, longitudes)
data_30km = load_data_at_altitude(30, data_folder_30km, latitudes, longitudes)
data_36km = load_data_at_altitude(36, data_folder_36km, latitudes, longitudes)
# data_42km = load_data_at_altitude(42, data_folder_42km, latitudes, longitudes)


# Combine data at both altitudes
combined_data = xr.concat([data_24km,data_30km,data_36km], dim='altitude')

# Create an xarray Dataset with a data variable named 'mfx'
combined_dataset = xr.Dataset({'mfx': combined_data})

# Save the combined dataset to a new .nc file
combined_dataset.to_netcdf('combined_data_with_coordinates.nc')

print('All data has been combined into a single .nc file with coordinates.')


All data has been combined into a single .nc file with coordinates.
